# 卷积神经网络
## 1.架构介绍
全连接神经网络的每一层都接收了上一层所有节点的信息，并且单层的神经元之间相互独立不共享任何参数。而卷积神经网络中使用了卷积运算，相比而言有三个重要特征，稀疏交互、参数共享和等变表示。
### 卷积运算
### 图像识别应用
MNIST的例子，输入层为\[32\*32\*3\]，使用全连接网络会有参数过多、计算代价高的问题，因此使用卷积网络，输出层为\[1\*1\*10\]对应10类得分。
![Full VS CNN](image/net.png)
"A ConvNet is made up of Layers. Every Layer has a simple API: It transforms an input 3D volume to an output 3D volume with some differentiable function that may or may not have parameters."
## 2.卷积网络层
结构：Input -- Conv -- Relu -- Pool -- FullC 
<br />1.Input:\[32\*32\*3\] （RGB）
<br />2.Conv:点乘权重和所连接的小区域，若选择12个filters则得到\[32\*32\*12\]。
<br />3.Relu:激活层，维度保持不变。
<br />4.Pool:对width和height缩减采样，得到\[16\*16\*12\]。
<br />5.FullC:计算每类得分，得到\[1\*1\*10\]
<br />Conv和FullC即包含激活部分也包含参数部分，参数由优化算法得到，Pool和Relu仅为一层固定函数。
![Car](image/car.png)
### 卷积层
卷积层参数由一系列可学习的过滤器（filters）组成，比如一个\[5\*5\*3\]的过滤器（depth保证和输入相同，width和height小于输入）。将单个过滤器遍历与原图相应位置点乘，得到2维的激活图，网络对该激活图学习，可得到如第一层的边缘信息、某些颜色的斑点和更高层的抽象图案等特征。假如有12个过滤器，每个对应一个2维的激活图，最后合在一起的维度\[32\*32\*12\]。
#### 稀疏连接(Local Connectivity)
过滤器的大小又称为接收域（如5×5），在图片识别中再次强调宽和高小于原始输入，深度等于原始输入也就是3。故卷积层的神经元的权重大小即为5×5×3=75（再加1个偏差）。
![Arch](image/arch.png)
#### 空间排列(Spatial arrangement)
输出卷积（volume）的排列涉及三个超参数，深度（depth）、步幅（stride）和补零法（zero_padding）
<br />1.depth:过滤器的个数，每个有不同的含义，比如那原始图片作为输入层，就可以有边缘识别、颜色判断等过滤器。depth column指的是不同神经元对应同一块输入区域。
<br />2.stride:遍历的步幅长度，影响输出卷积的大小。
<br />3.zero-padding:边缘补0，控制输出大小
![size](image/size.png)
#### 参数共享(Parameter Sharing)
对于size为\[55\*55\*96\]的卷（volume），有96个depth slices（\[55\*55\]），参数共享指的是同一slice的神经元（neuron）使用相同的权重和偏差，固整体参数个数为96\*11\*11\*3=34848,+96biases。后馈计算时，会按slice整体更新参数。由于参数共享的机制，每一层的计算实际上是神经元的权重和输入卷的卷积（convolution），过滤器（filter）也称为卷积中的核函数（kernel）。
<br />有一些情况参数共享不适用，如果原图的识别受位置影响，如脸部识别问题，这时需要采用局部连接层方法（Locally-Connected Layer）
![sum](image/sum.png)

#### 卷积过程的可视化      
            
<img src="gif/demo.gif">

#### Padding的可视化过程
+ padding的形式："VALID","SAME":
    + VALID：比较容易理解，filter全部在image里面
        + SAME：满足$n_{out}=\lceil \frac {n_{in}} {s}\rceil$，即当步长s为1时，大小保持不变；各个方向补充0的规则为
            + $pad_h = max[( n_{out} -1 ) × s_h + f_h - n_{in} ， 0]$
            + $pad_{top} = \lfloor pad_h / 2 \rfloor$  # 注意此处向下取整
            + $pad_{bottom} = pad_h - pad_{top}$
            + $pad_w = max[( n_{out} -1 ) × s_w + f_w - n_{in} ， 0]$
            + $pad_{left} = \lfloor pad_w / 2 \rfloor$  # 注意此处向下取整
            + $pad_{right} = pad_w - pad_{left}$
+ [padding的图解](https://blog.csdn.net/leviopku/article/details/80327478)
            
<table style="width:85%; table-layout:fixed;">
  <tr>
    <td><img width="180px" src="gif/no_padding_no_strides.gif"></td>
    <td><img width="180px" src="gif/arbitrary_padding_no_strides.gif"></td>
    <td><img width="180px" src="gif/same_padding_no_strides.gif"></td>
    <td><img width="180px" src="gif/full_padding_no_strides.gif"></td>
  </tr>
  <tr>
    <td>No padding, no strides</td>
    <td>Arbitrary padding, no strides</td>
    <td>Half padding, no strides</td>
    <td>Full padding, no strides</td>
  </tr>
  <tr>
    <td><img width="180px" src="gif/no_padding_strides.gif"></td>
    <td><img width="180px" src="gif/padding_strides.gif"></td>
    <td><img width="180px" src="gif/padding_strides_odd.gif"></td>
    <td></td>
  </tr>
  <tr>
    <td>No padding, strides</td>
    <td>Padding, strides</td>
    <td>Padding, strides (odd)</td>
    <td></td>
  </tr>
</table>


### 池化层
此层的作用是逐渐减少空间大小以减少参数和相应计算复杂度，同时也可以控制过拟合的问题。
![pool1](image/pool1.png)
![pool2](image/pool2.png)
一些情况池化层并不好用，可以通过增大步幅的方式代替池化减少size的方法。丢弃池化层在一些问题里是很重要的，如变化的自编码器、生成对抗网络（GANS)等。
### 全连接层
&emsp;&emsp;在卷积部分和最后输入层之间，一般还会加入若干fully-connected/dense layer(s)。<br>
&emsp;&emsp;若卷积最后输出tensor的shape为`[w,h,c]`在进行全连接前需要reshape为向量形式，长度为`w*h*c`。<br>
&emsp;&emsp;<font color="red">那么为什么还需要在输出层之前添加全连接层？</font>
- __多个卷积层叠加后的tensor形状导致的自然选择。__观察下图
![LeNet model](image/lenet.jpg)
可以发现图像的3维结构由“宽且薄”的扁平型逐渐变为“窄且厚”的细长型。在极端情况下，最后一层卷积（或池化）输出的图像切面仅剩一个像素，维度变为`[1,1,d]`，这就是全连接层的输入形式。这表明，在池化的过程中，切面（width和height维度）上的信息逐渐转化到通道(depth，第3维度)上，特征不断被抽象出来，同时丧失了它作为图像的属性(2D结构)。因此更应该被视作是向量而不是图像。
- __卷积运算和全连接层可以相互转换__：他们本质上都是_dot products_！<br>
  - 卷积层表示的连接，可以扩展为一个维度很大但稀疏（局部连接性）的矩阵，并且很多分块稠密的权重是相等（参数共享）的全连接层。
  - 长度为K的全连接层的输出可以被看做是`[1,1,K]`的3维tensor，它可由一个卷积层得到。假设这个卷积层的输入维度为`[w,w,p]`，那么卷积核维度与输入相同，并拥有K个卷积核。
  <img src="image/lecun.jpg" height="80%" width="80%"></img>
- __全连接层包含较多参数，能够提升模型容量__：这与多层感知机中增加层数的解释相同，是对提取特征的进一步非线性变换和抽象。
- __加入全连接层不是必须做法__：[部分文献](https://cs.stanford.edu/~quocle/LeSarlosSmola_ICML13.pdf")用fastfood layer替代fully-connected layer，减少了冗余变量的同时也取得了很好的表现。


## 3.卷积网络架构
### 分层模式
&emsp;&emsp;较经典的卷积网络结构：INPUT -> \[[CONV -> RELU]\*N -> POOL?\]\*M -> \[FC -> RELU\]\*K -> FC<br />
&emsp;&emsp;超越AlexNet的VGG网络，重要改进是大量使用3\*3的小卷积核，并加深卷积层的堆叠。因为2个3\*3的卷积层接受野(receptive field)等价一个5\*5的卷积核；3个3\*3的卷积层堆叠的接受野等价于1个7\*7的卷积核，如下图：
<img src="image/vgg.jpg" height="60%" width="60%"></img>
并且更深层的卷积堆积能够带来更多的非线性以提升模型容量，并且获得参数个数降低的正则化效果。
<br /> &emsp;&emsp;近年，传统的线性列表结构受到了挑战，未来更复杂更不同的连接结构更被倾向于使用，我们实战时可以参考类似于ImageNet等已有的网络结构，根据自己的数据进行调整。

### 尺寸的一般选择(size patterns)
&emsp;&emsp;卷积网络各层size的选择是超参数，但一般遵循一些模式：
- __input layer__：应当尽量能被2多次整除，常用的尺寸包括32,64，96，224等
- __conv layer__：卷积核size一般较小（3\*3，不超过5*5）；在准备设计加入pool layer的时候，步长stride一般选择为1；并且应当选择适当的padding(补0单元)保持卷积后图像的尺寸。
- __pool layer__：池化层一般选择最大池化，且filter/kernel为2\*2，stride=2。一般池化的size不会超过3，因为池化层会破坏图像的细微结构并丢失大量信息。